# Load Bloomberg data from Excel 

Data has been downloaded via VBA Script and saved on BloombergDataCleaned.xlsm
No time series gap filling / or blanding have been applied. Data Cleaning will be done here in Python.

In [ ]:
import pandas as pd
import datetime 

In [161]:
# some data is saved in different Sheets
data4 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet4', index_col='Timestamp')
data5 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet5', index_col='Timestamp')
data6 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet6', index_col='Timestamp')
data7 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet7', index_col='Timestamp')
data8 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet8', index_col='Timestamp')
data9 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet9', index_col='Timestamp')
data10 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet10', index_col='Timestamp')
data11 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet11', index_col='Timestamp')
data12 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet12', index_col='Timestamp')
data13 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet13', index_col='Timestamp')
data14 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet14', index_col='Timestamp')
data15 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet15', index_col='Timestamp')

In [162]:
# align the timezone
data5.index = data5.index + pd.Timedelta(hours=1)
data13.index = data13.index + pd.Timedelta(hours=1)
data14.index = data14.index + pd.Timedelta(hours=1)
data15.index = data15.index + pd.Timedelta(hours=1)

'''# reindex to align the timezone
data[3].index = data[3].index + pd.Timedelta(hours=1)
data[5].index = data[5].index + pd.Timedelta(hours=1)
data[6].index = data[6].index + pd.Timedelta(hours=1)
data[10].index = data[10].index + pd.Timedelta(hours=1)
data[18].index = data[18].index + pd.Timedelta(hours=1)
data[22].index = data[22].index + pd.Timedelta(hours=1)
data[25].index = data[25].index + pd.Timedelta(hours=1)
'''
# you could also do it dynamically using 
''' 
for i in range(0, 36):
    if data[i].index[0].time().hour == 7:
        data[i].index = data[i].index + pd.Timedelta(hours=1)
'''

In [163]:
# more data
data = dict()

j = 0
for i in range(0, 107, 3):
    # read the excel file
    data[j] = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet0', parse_cols=[i,i+1], index_col=0)
    # drop nan
    data[j].dropna(inplace=True)
    # reindex to align the timezone
    if data[j].index[0].time().hour == 7:
        data[j].index = data[j].index + pd.Timedelta(hours=1)
    j = j+1

In [164]:
# generate the index manually. Otherwise standard resampling methods would include all the 
# hours of the day! We do care only about trading hours and Business Days (freq='B')
index_date = pd.date_range('2018-01-02', '2018-02-15', freq='B')
index_date = pd.Series(index_date)
index_time = pd.date_range('08:00:00', '16:40:00', freq='1min')
index_time = pd.Series(index_time.time)

index = index_date.apply(
    lambda d: index_time.apply(
        lambda t: datetime.datetime.combine(d, t)
        )
    ).unstack().sort_values().reset_index(drop=True)

print(index.shape)

(17193,)


In [165]:
# reindex the Series with the newly generated index
data4 = data4.reindex(index=index)
data5 = data5.reindex(index=index)
data6 = data6.reindex(index=index)
data7 = data7.reindex(index=index)
data8 = data8.reindex(index=index)
data9 = data9.reindex(index=index)
data10 = data10.reindex(index=index)
data11 = data11.reindex(index=index)
data12 = data12.reindex(index=index)
data13 = data13.reindex(index=index)
data14 = data14.reindex(index=index)
data15 = data15.reindex(index=index)

# interpolate to get the values for the NaNs
data4 = data4.interpolate(method='linear')
data5 = data5.interpolate(method='linear')
data6 = data6.interpolate(method='linear')
data7 = data7.interpolate(method='linear')
data8 = data8.interpolate(method='linear')
data9 = data9.interpolate(method='linear')
data10 = data10.interpolate(method='linear')
data11 = data11.interpolate(method='linear')
data12 = data12.interpolate(method='linear')
data13 = data13.interpolate(method='linear')
data14 = data14.interpolate(method='linear')
data15 = data15.interpolate(method='linear')

In [ ]:
# move everything into a single dictionary
data[36] = data4
data[37] = data5
data[38] = data6
data[39] = data7
data[40] = data8
data[41] = data9
data[42] = data10
data[43] = data11
data[44] = data12
data[45] = data13
data[46] = data14
data[47] = data15

In [228]:
# create an empty df indexed like my data
data_open = pd.DataFrame(index=index)

# join all the dictionary items into one single df
for k in data.keys():
    data_open = data_open.join(data[k])
print(data_open.shape)

(17193, 48)


In [229]:
# save the data into a csv file 
data_open.to_csv('closing_prices_cleaned.csv')

# End of data preparation 